In [1]:
%load_ext autoreload
%autoreload 2

import os
import numpy as np
import pandas as pd 
import sys
import pickle
import itertools
import seaborn as sns
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import random
from sklearn.metrics import pairwise_distances
from sklearn.kernel_ridge import KernelRidge
import math

workingdirectory = os.popen('git rev-parse --show-toplevel').read()[:-1]
sys.path.append(workingdirectory)
os.chdir(workingdirectory)

import allensdk.core.json_utilities as ju
from allensdk.core.mouse_connectivity_cache import MouseConnectivityCache

from mcmodels.core import VoxelModelCache
from mcmodels.core.utils import get_structure_id,get_ordered_summary_structures
from mcmodels.core.connectivity_data import get_connectivity_data
from mcmodels.models.crossvalidation import get_nwloocv_predictions_multimodel_merge_dists
from mcmodels.utils import nonzero_unique #, unionize
from mcmodels.models.crossvalidation import get_best_hyperparameters,get_loss_best_hyp,get_loss#get_loocv_predictions,get_loss#get_best_hyperparameters,get_loss_best_hyp,get_loocv_predictions,get_loss
from mcmodels.core.utils import get_leaves_ontologicalorder, get_indices, get_indices2,get_eval_indices,screen_index_matrices,screen_index_matrices2,screen_index_matrices3#get_cre_status,get_minorstructure_dictionary,get_leaves_ontologicalorder
from mcmodels.core.utils import get_indices_2ormore

# from mcmodels.core.utils import get_regionalized_normalized_data
# from mcmodels.core.utils import get_connectivity
# from mcmodels.core.utils import get_ontological_order_leaf
# from mcmodels.core.utils import get_nw_loocv,get_wt_inds
# from mcmodels.core.utils import get_countvec
# from mcmodels.core.utils import get_injection_hemisphere_id
# from mcmodels.core.utils import get_structure_id, get_ordered_summary_structures,get_minorstructures,get_loss_paper
# from mcmodels.core.experiment import get_voxeldata_msvd
# from mcmodels.regressors import NadarayaWatson

/Users/samsonkoelle/anaconda3/envs/allen_010719_5/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.linear_model.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.linear_model. Anything that cannot be imported from sklearn.linear_model is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/samsonkoelle/anaconda3/envs/allen_010719_5/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
#read data
TOP_DIR = '/Users/samsonkoelle/alleninstitute/mcm_2020/mcm_updated/'
INPUT_JSON = os.path.join(TOP_DIR, 'input_011520.json')
EXPERIMENTS_EXCLUDE_JSON = os.path.join(TOP_DIR, 'experiments_exclude.json')
FILE_DIR = '/Users/samsonkoelle/alleninstitute/mcm_2020/mcm_updated/'
OUTPUT_DIR = os.path.join(FILE_DIR, 'output')

input_data = ju.read(INPUT_JSON)
manifest_file = input_data.get('manifest_file')
manifest_file = os.path.join(TOP_DIR, manifest_file)
experiments_exclude = ju.read(EXPERIMENTS_EXCLUDE_JSON)

#its unclear why the hyperparameters are loaded from the output directory
cache = VoxelModelCache(manifest_file=manifest_file)
major_structures = input_data.get('structures')
major_structure_ids = [get_structure_id(cache, s) for s in major_structures]
data_info = pd.read_excel('/Users/samsonkoelle/alleninstitute/Whole Brain Cre Image Series_curation only.xlsx', 'all datasets curated_070919pull')
data_info.set_index("id", inplace=True)
ontological_order = get_ordered_summary_structures(cache)

mcc = MouseConnectivityCache(manifest_file = '../connectivity/mouse_connectivity_manifest.json')
st = mcc.get_structure_tree()
ai_map = st.get_id_acronym_map()
ia_map = {value: key for key, value in ai_map.items()}

#regionalize voxel model: compare with regional model
#regional parameters
cre = None
eid_set=None
high_res=False
threshold_injection = False

COARSE_STRUCTURE_SET_ID = 2
DEFAULT_STRUCTURE_SET_IDS = tuple([COARSE_STRUCTURE_SET_ID])
tree = cache.get_structure_tree()
default_structures = tree.get_structures_by_set_id(DEFAULT_STRUCTURE_SET_IDS)
default_structure_ids = [st['id'] for st in default_structures if st['id'] != 934]
#cre= True

In [3]:
connectivity_data = get_connectivity_data(cache, major_structure_ids[:12], experiments_exclude, remove_injection = False)

512
703
1089
1097
315
313
354
698
771
803
477
549


In [8]:
connectivity_data.get_injection_hemisphere_ids()
connectivity_data.align()
connectivity_data.get_centroids()
connectivity_data.get_data_matrices(default_structure_ids)
connectivity_data.get_crelines(data_info)

In [9]:
with open('data/info/leafs.pickle', 'rb') as handle:
    leafs = pickle.load(handle)
    
connectivity_data.ai_map = ai_map

In [10]:
connectivity_data.get_summarystructures(data_info)
connectivity_data.leafs = leafs
#connectivity_data.get_leafs(leafs)

In [11]:
#major division segregation is legacy code but convenient for fast cross validation in major division model
#experiments_minor_structures = get_summarystructure_dictionary(connectivity_data, data_info)
#get leaves in ontological order.  Where leafs don't exist, uses summary structure
ontological_order_leaves = get_leaves_ontologicalorder(connectivity_data, ontological_order)
#Key isn't affected by which experiment we choose. This allows default masking to be inherited from the AllenSDK.
sid0 = list(connectivity_data.structure_datas.keys())[0]
eid0 = list(connectivity_data.structure_datas[sid0].experiment_datas.keys())[0]
#Identify keys denoting which voxels correspond to which structure in the ipsi and contra targets.
#contra_targetkey = msvd.experiments[list(msvd.experiments.keys())[0]].projection_mask.get_key(structure_ids=ontological_order_leaves, hemisphere_id=1)
#ipsi_targetkey = msvd.experiments[list(msvd.experiments.keys())[0]].projection_mask.get_key(structure_ids=ontological_order_leaves, hemisphere_id=2)
contra_targetkey = connectivity_data.structure_datas[sid0].projection_mask.get_key(structure_ids=ontological_order, hemisphere_id=1)
ipsi_targetkey = connectivity_data.structure_datas[sid0].projection_mask.get_key(structure_ids=ontological_order, hemisphere_id=2)
#get average intensities of projection structures given ipsi and contra keys
#source_key = ontological_order #only relevant here when injection needs to be unionized, but currently a required argument
ipsi_target_regions, ipsi_target_counts = nonzero_unique(ipsi_targetkey, return_counts=True)
contra_target_regions, contra_target_counts = nonzero_unique(contra_targetkey, return_counts=True)

target_order = lambda x: np.array(ontological_order)[np.isin(ontological_order, x)]
permutation = lambda x: np.argsort(np.argsort(target_order(x)))
targ_ids = np.concatenate([ipsi_target_regions[permutation(ipsi_target_regions)],
                           contra_target_regions[permutation(contra_target_regions)]])

In [12]:
connectivity_data.get_regionalized_normalized_data(ontological_order, ipsi_targetkey, contra_targetkey)

In [13]:
connectivity_data.get_creleaf_combos()

In [14]:
connectivity_data.leaf2_index_matrices = get_indices_2ormore(connectivity_data.leafs)
connectivity_data.creleaf2_index_matrices = get_indices_2ormore(connectivity_data.creleaf_combos)
connectivity_data.creleaf2_evalindices = get_eval_indices(connectivity_data.creleaf2_index_matrices)


In [16]:
#we want to find the model rows that include the eval rows
def get_matched_index_matrices(model_index_matrices, eval_index_matrices):
    
    major_structure_ids = np.asarray(list(eval_index_matrices.keys()))
    matched_model_index_matrices = {}
    #eval_indices = get_eval_indices(eval_index_matrices)
    for sid in major_structure_ids:
        matched_model_index_matrices[sid ] = np.zeros(eval_index_matrices[sid].shape)
#         model_indices = []
#         for m in range(model_index_matrices[sid].shape[0]):
#             model_indices = model_indices.append(np.where(model_index_matrices[sid] == 1)[0])
        for m in range(eval_index_matrices[sid].shape[0]):
            for n in range(model_index_matrices[sid].shape[0]):
                model_indices = np.where(model_index_matrices[sid][n]  == 1)[0]
                eval_indices = np.where(eval_index_matrices[sid][m]  == 1)[0]
                if len(np.intersect1d(model_indices,eval_indices)) > 0:
                    #difference = model_index_matrices[n] - eval_index_matrices[m]
                    #if np.min(difference != -1):
                    print(model_index_matrices[sid][n] .sum())
                    matched_model_index_matrices[sid ][m] = model_index_matrices[sid][n]        
        matched_model_index_matrices[sid] = np.asarray(matched_model_index_matrices[sid], dtype = int)
    #print(matched_model_index_matrices)
    return(matched_model_index_matrices)  

    

In [17]:
connectivity_data.leaf2_creleaf2_index_matrices = get_matched_index_matrices(connectivity_data.leaf2_index_matrices,connectivity_data.creleaf2_index_matrices)

6
9
2
2
8
8
12
12
18
18
18
18
21
21
21
21
6
8
8
10
10
8
9
9
5
4
8
3
9
9
7
7
4
4
3
3
11
11
11
10
10
4
3
41
41
41
41
41
41
41
41
41
41
41
41
41
12
12
8
8
3
20
20
20
20
4
3
29
29
29
29
29
29
29
29
29
18
18
18
18
18
11
11
11
9
6
6
12
9
9
9
5
6
14
14
14
14
5
4
7
3
12
12
12
12
9
9
9
9
6
6
11
11
11
11
24
24
24
24
24
24
9
9
9
26
26
26
26
26
10
3
6
6
22
22
22
22
22
3
2
20
20
20
20
20
20
14
14
14
14
14
7
9
9
9
9
23
23
23
23
23
23
23
23
8
8
13
23
23
23
23
23
4
23
23
23
23
23
23
13
75
75
75
75
75
75
75
75
75
75
6
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
33
33
33
33
33
33
33
33
33
13
13
13
13
13
124
124
124
124
124
124
124
124
124
124
124
124
124
9
9
18
18
18
7
3
25
25
25
10
10
10
10
6
8
8
8
7
7
24
24
24
24
24
12
12
12
3
5
15
15
6
3
6
5
5
6
5
8
8
14
14
14
12
3
3
2
4
6
8
4
4
7
9
8
8
43
43
43
43
43
43
43
43
3
4
5
4
5
5
5
11
11
11
6
6
3


In [18]:
connectivity_data.get_pairwise_distances()

In [19]:
gammas = np.asarray([0.1,.5,1,2,10,20,40])

In [20]:
#major_structure_ids
for sid in np.asarray(list(connectivity_data.structure_datas.keys())):
    connectivity_data.structure_datas[sid].loocv_predictions_leaf2_creleaf2 = get_nwloocv_predictions_multimodel_merge_dists(connectivity_data.structure_datas[sid].reg_proj_norm, 
                                                                                       connectivity_data.structure_datas[sid].pairwise_distances,#np.ones(losses.shape),#(losses - np.nanmin(losses))**2,#**6, 
                                                                                       gammas, 
                                                                                       connectivity_data.leaf2_index_matrices[sid], 
                                                                                       connectivity_data.leaf2_index_matrices[sid])                                  

#meanloss_nw_leaf_leaf2wt
#meanloss_nw_wtleaf_leaf2wt

e [ 0 12 16] m [ 0 12 16]
e [ 0 12 16] m [ 0 12 16]
e [ 0 12 16] m [ 0 12 16]
e [ 0 12 16] m [ 0 12 16]
e [ 0 12 16] m [ 0 12 16]
e [ 0 12 16] m [ 0 12 16]
e [ 0 12 16] m [ 0 12 16]
m 0 3 0.003720867 0.008055458025002557
e [ 6 28] m [ 6 28]
e [ 6 28] m [ 6 28]
e [ 6 28] m [ 6 28]
e [ 6 28] m [ 6 28]
e [ 6 28] m [ 6 28]
e [ 6 28] m [ 6 28]
e [ 6 28] m [ 6 28]
m 1 2 0.003720867 0.008605728374282284
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 2 0 0.003720867 nan
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 3 0 0.003720867 nan
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 4 0 0.003720867 nan
e [ 9 19] m [ 9 19]
e [ 9 19] m [ 9 19]
e [ 9 19] m [ 9 19]
e [ 9 19] m [ 9 19]
e [ 9 19] m [ 9 19]
e [ 9 19] m [ 9 19]
e [ 9 19] m [ 9 19]
m 5 2 0.003720867 0.003088656711986914
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 6 0 0.003720867 nan
e [ 5  7 14 15 30 31] m [ 5  7 14 15 30 31]
e [ 5  7 

/Users/samsonkoelle/alleninstitute/sambranch/mouse_connectivity_models/mcmodels/models/crossvalidation.py:143: RuntimeWarning: Mean of empty slice
  print('m', m, len(np.where(model_index_matrix[m] == 1)[0]), np.nanmean(projections), np.nanmean(predictions[m]))
/Users/samsonkoelle/alleninstitute/sambranch/mouse_connectivity_models/mcmodels/models/crossvalidation.py:107: RuntimeWarning: overflow encountered in exp
  weights_i = np.exp(-dists_i * gamma)  # weights[i,:] / np.nansum(weights[i,:][otherindices])


m 0 0 0.004429725 nan
e [ 50  88 113] m [ 50  88 113]
e [ 50  88 113] m [ 50  88 113]
e [ 50  88 113] m [ 50  88 113]
e [ 50  88 113] m [ 50  88 113]
e [ 50  88 113] m [ 50  88 113]
e [ 50  88 113] m [ 50  88 113]
e [ 50  88 113] m [ 50  88 113]
m 1 3 0.004429725 0.00413989358000956
e [  3  13  44  45  49  61  68 120] m [  3  13  44  45  49  61  68 120]
e [  3  13  44  45  49  61  68 120] m [  3  13  44  45  49  61  68 120]
e [  3  13  44  45  49  61  68 120] m [  3  13  44  45  49  61  68 120]
e [  3  13  44  45  49  61  68 120] m [  3  13  44  45  49  61  68 120]
e [  3  13  44  45  49  61  68 120] m [  3  13  44  45  49  61  68 120]
e [  3  13  44  45  49  61  68 120] m [  3  13  44  45  49  61  68 120]
e [  3  13  44  45  49  61  68 120] m [  3  13  44  45  49  61  68 120]
m 2 8 0.004429725 0.005631958358752307
e [  0  32  37  38  51  52  66  70  86  91 103 108] m [  0  32  37  38  51  52  66  70  86  91 103 108]
e [  0  32  37  38  51  52  66  70  86  91 103 108] m [  0  32  37  3

e [15 42 61 72] m [15 42 61 72]
e [15 42 61 72] m [15 42 61 72]
m 11 4 0.008953513 0.0050881821291617665
e [38 59] m [38 59]
e [38 59] m [38 59]
e [38 59] m [38 59]
e [38 59] m [38 59]
e [38 59] m [38 59]
e [38 59] m [38 59]
e [38 59] m [38 59]
m 12 2 0.008953513 0.004332190723298556
e [34 69] m [34 69]
e [34 69] m [34 69]
e [34 69] m [34 69]
e [34 69] m [34 69]
e [34 69] m [34 69]
e [34 69] m [34 69]
e [34 69] m [34 69]
m 13 2 0.008953513 0.007258814402142294
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 14 0 0.008953513 nan
e [ 9 21 78] m [ 9 21 78]
e [ 9 21 78] m [ 9 21 78]
e [ 9 21 78] m [ 9 21 78]
e [ 9 21 78] m [ 9 21 78]
e [ 9 21 78] m [ 9 21 78]
e [ 9 21 78] m [ 9 21 78]
e [ 9 21 78] m [ 9 21 78]
m 15 3 0.008953513 0.008356481582483276
e [ 0  4 10 22 29 33 35 51 57 58 75] m [ 0  4 10 22 29 33 35 51 57 58 75]
e [ 0  4 10 22 29 33 35 51 57 58 75] m [ 0  4 10 22 29 33 35 51 57 58 75]
e [ 0  4 10 22 29 33 35 51 57 58 75] m [ 0  4 10 22 29 33 35 51 57 58 75

m 13 0 0.0051781153 nan
e [690 821] m [690 821]
e [690 821] m [690 821]
e [690 821] m [690 821]
e [690 821] m [690 821]
e [690 821] m [690 821]
e [690 821] m [690 821]
e [690 821] m [690 821]
m 14 2 0.0051781153 0.003836928191182778
e [796 884] m [796 884]
e [796 884] m [796 884]
e [796 884] m [796 884]
e [796 884] m [796 884]
e [796 884] m [796 884]
e [796 884] m [796 884]
e [796 884] m [796 884]
m 15 2 0.0051781153 0.005000795136673242
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 16 0 0.0051781153 nan
e [ 46 460 567] m [ 46 460 567]
e [ 46 460 567] m [ 46 460 567]
e [ 46 460 567] m [ 46 460 567]
e [ 46 460 567] m [ 46 460 567]
e [ 46 460 567] m [ 46 460 567]
e [ 46 460 567] m [ 46 460 567]
e [ 46 460 567] m [ 46 460 567]
m 17 3 0.0051781153 0.007749948404476363
e [102 400] m [102 400]
e [102 400] m [102 400]
e [102 400] m [102 400]
e [102 400] m [102 400]
e [102 400] m [102 400]
e [102 400] m [102 400]
e [102 400] m [102 400]
m 18 2 0.0051781153 0.007163358

...

m 91 3 0.0051781153 0.00676001301845712
e [  31   32   44  128  234  264  281  330  365  385  388  408  435  456
  472  483  538  552  565  588  622  820  873  955  977  991 1033 1034
 1077] m [  31   32   44  128  234  264  281  330  365  385  388  408  435  456
  472  483  538  552  565  588  622  820  873  955  977  991 1033 1034
 1077]
e [  31   32   44  128  234  264  281  330  365  385  388  408  435  456
  472  483  538  552  565  588  622  820  873  955  977  991 1033 1034
 1077] m [  31   32   44  128  234  264  281  330  365  385  388  408  435  456
  472  483  538  552  565  588  622  820  873  955  977  991 1033 1034
 1077]
e [  31   32   44  128  234  264  281  330  365  385  388  408  435  456
  472  483  538  552  565  588  622  820  873  955  977  991 1033 1034
 1077] m [  31   32   44  128  234  264  281  330  365  385  388  408  435  456
  472  483  538  552  565  588  622  820  873  955  977  991 1033 1034
 1077]
e [  31   32   44  128  234  264  281  330  365  385  

e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 103 0 0.0051781153 nan
e [172 542] m [172 542]
e [172 542] m [172 542]
e [172 542] m [172 542]
e [172 542] m [172 542]
e [172 542] m [172 542]
e [172 542] m [172 542]
e [172 542] m [172 542]
m 104 2 0.0051781153 0.006983677917283166
e [ 139  180  205  215  218  352  359  474  822  925 1018 1054] m [ 139  180  205  215  218  352  359  474  822  925 1018 1054]
e [ 139  180  205  215  218  352  359  474  822  925 1018 1054] m [ 139  180  205  215  218  352  359  474  822  925 1018 1054]
e [ 139  180  205  215  218  352  359  474  822  925 1018 1054] m [ 139  180  205  215  218  352  359  474  822  925 1018 1054]
e [ 139  180  205  215  218  352  359  474  822  925 1018 1054] m [ 139  180  205  215  218  352  359  474  822  925 1018 1054]
e [ 139  180  205  215  218  352  359  474  822  925 1018 1054] m [ 139  180  205  215  218  352  359  474  822  925 1018 1054]
e [ 139  180  205  215  218  352  359  474  822  925 1018 1054] m

e [10 18] m [10 18]
e [10 18] m [10 18]
e [10 18] m [10 18]
e [10 18] m [10 18]
e [10 18] m [10 18]
e [10 18] m [10 18]
e [10 18] m [10 18]
m 0 2 0.007270674 0.007986724959016782
e [12 14 20 35 41] m [12 14 20 35 41]
e [12 14 20 35 41] m [12 14 20 35 41]
e [12 14 20 35 41] m [12 14 20 35 41]
e [12 14 20 35 41] m [12 14 20 35 41]
e [12 14 20 35 41] m [12 14 20 35 41]
e [12 14 20 35 41] m [12 14 20 35 41]
e [12 14 20 35 41] m [12 14 20 35 41]
m 1 5 0.007270674 0.005134984841221223
e [28 31] m [28 31]
e [28 31] m [28 31]
e [28 31] m [28 31]
e [28 31] m [28 31]
e [28 31] m [28 31]
e [28 31] m [28 31]
e [28 31] m [28 31]
m 2 2 0.007270674 0.005305797807702007
e [15 26] m [15 26]
e [15 26] m [15 26]
e [15 26] m [15 26]
e [15 26] m [15 26]
e [15 26] m [15 26]
e [15 26] m [15 26]
e [15 26] m [15 26]
m 3 2 0.007270674 0.0059811264294997776
e [ 6 34 43] m [ 6 34 43]
e [ 6 34 43] m [ 6 34 43]
e [ 6 34 43] m [ 6 34 43]
e [ 6 34 43] m [ 6 34 43]
e [ 6 34 43] m [ 6 34 43]
e [ 6 34 43] m [ 6 34 43]
e

e [26 27 32 48 54 55 70 74] m [26 27 32 48 54 55 70 74]
e [26 27 32 48 54 55 70 74] m [26 27 32 48 54 55 70 74]
m 0 8 0.0038424605 0.003810396901743683
e [14 24 34 65] m [14 24 34 65]
e [14 24 34 65] m [14 24 34 65]
e [14 24 34 65] m [14 24 34 65]
e [14 24 34 65] m [14 24 34 65]
e [14 24 34 65] m [14 24 34 65]
e [14 24 34 65] m [14 24 34 65]
e [14 24 34 65] m [14 24 34 65]
m 1 4 0.0038424605 0.004123446969852839
e [12 19 31 38 39 53 60] m [12 19 31 38 39 53 60]
e [12 19 31 38 39 53 60] m [12 19 31 38 39 53 60]
e [12 19 31 38 39 53 60] m [12 19 31 38 39 53 60]
e [12 19 31 38 39 53 60] m [12 19 31 38 39 53 60]
e [12 19 31 38 39 53 60] m [12 19 31 38 39 53 60]
e [12 19 31 38 39 53 60] m [12 19 31 38 39 53 60]
e [12 19 31 38 39 53 60] m [12 19 31 38 39 53 60]
m 2 7 0.0038424605 0.005048372257250426
e [ 3 10 17 20 35 43 45 57 59] m [ 3 10 17 20 35 43 45 57 59]
e [ 3 10 17 20 35 43 45 57 59] m [ 3 10 17 20 35 43 45 57 59]
e [ 3 10 17 20 35 43 45 57 59] m [ 3 10 17 20 35 43 45 57 59]
e [ 3 10

e [27 59 71] m [27 59 71]
e [27 59 71] m [27 59 71]
e [27 59 71] m [27 59 71]
m 27 3 0.005375255 0.0028541959812177116
e [ 7 16] m [ 7 16]
e [ 7 16] m [ 7 16]
e [ 7 16] m [ 7 16]
e [ 7 16] m [ 7 16]
e [ 7 16] m [ 7 16]
e [ 7 16] m [ 7 16]
e [ 7 16] m [ 7 16]
m 28 2 0.005375255 0.007852306496526249
(29, 7, 83, 577)


In [22]:
keys = np.asarray(list(itertools.product(*[list(range(len(gammas)))])))
reg_proj_norm = {}
nwloocv_leaf2_creleaf2 = {}
for sid in np.asarray(list(connectivity_data.structure_datas.keys())):
    reg_proj_norm[sid] = connectivity_data.structure_datas[sid].reg_proj_norm
    nwloocv_leaf2_creleaf2[sid] = connectivity_data.structure_datas[sid].loocv_predictions_leaf2_creleaf2

losses_leaf2_creleaf2 = get_loss(reg_proj_norm, nwloocv_leaf2_creleaf2,pred_ind = connectivity_data.creleaf2_evalindices, true_ind = connectivity_data.creleaf2_evalindices,keys = keys)
best_gamma_leaf2_creleaf2 = get_best_hyperparameters(losses_leaf2_creleaf2,keys)
meanloss_nw_leaf2_creleaf2 = get_loss_best_hyp(losses_leaf2_creleaf2, best_gamma_leaf2_creleaf2)

meanloss_nw_leaf2_creleaf2

0
1
2
3
4
5
6
7
8
9
10
11


array([0.06492248, 0.4966726 , 0.12877972, 0.23367772, 0.18424232,
       0.12156122, 0.19349936, 0.07775228, 0.25703143, 0.25605966,
       0.08174782, 0.31461163])

In [177]:
#major_structure_ids
for sid in np.asarray(list(connectivity_data.structure_datas.keys())):
    connectivity_data.structure_datas[sid].loocv_predictions_leaf2_creleaf2 = get_nwloocv_predictions_multimodel_merge_dists(connectivity_data.structure_datas[sid].reg_proj_norm, 
                                                                                       connectivity_data.structure_datas[sid].pairwise_distances,#np.ones(losses.shape),#(losses - np.nanmin(losses))**2,#**6, 
                                                                                       gammas, 
                                                                                       connectivity_data.leaf2_creleaf2_index_matrices[sid], 
                                                                                       connectivity_data.creleaf2_index_matrices[sid])                                  

#meanloss_nw_leaf_leaf2wt
#meanloss_nw_wtleaf_leaf2wt

e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 0 0 0.003720867 nan
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 1 0 0.003720867 nan
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 2 0 0.003720867 nan
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 3 0 0.003720867 nan
e [ 7 31] m [ 5  7 14 15 30 31]
e [ 7 31] m [ 5  7 14 15 30 31]
e [ 7 31] m [ 5  7 14 15 30 31]
e [ 7 31] m [ 5  7 14 15 30 31]
e [ 7 31] m [ 5  7 14 15 30 31]
e [ 7 31] m [ 5  7 14 15 30 31]
e [ 7 31] m [ 5  7 14 15 30 31]
m 4 6 0.003720867 0.0021063694794153758
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 5 0 0.003720867 nan
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 6 0 0.003720867 nan
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 7 0 0.003720867 nan
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 8 0 0.003720867 nan


/Users/samsonkoelle/anaconda3/envs/allen_010719_5/lib/python3.7/site-packages/ipykernel_launcher.py:68: RuntimeWarning: Mean of empty slice
/Users/samsonkoelle/anaconda3/envs/allen_010719_5/lib/python3.7/site-packages/ipykernel_launcher.py:31: RuntimeWarning: overflow encountered in exp


e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 27 0 0.003720867 nan
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 28 0 0.003720867 nan
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 29 0 0.003720867 nan
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 30 0 0.003720867 nan
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 31 0 0.003720867 nan
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 32 0 0.003720867 nan
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 33 0 0.003720867 nan
(34, 7, 36, 577)
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 0 0 0.006630151 nan
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 1 0 0.006630151 nan
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 2 0 0.006630151 nan
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m

m 45 0 0.004429725 nan
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 46 0 0.004429725 nan
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 47 0 0.004429725 nan
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 48 0 0.004429725 nan
e [ 4 26] m [  2   4  12  26  48 111]
e [ 4 26] m [  2   4  12  26  48 111]
e [ 4 26] m [  2   4  12  26  48 111]
e [ 4 26] m [  2   4  12  26  48 111]
e [ 4 26] m [  2   4  12  26  48 111]
e [ 4 26] m [  2   4  12  26  48 111]
e [ 4 26] m [  2   4  12  26  48 111]
m 49 6 0.004429725 0.005150654667178294
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 50 0 0.004429725 nan
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 51 0 0.004429725 nan
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 52 0 0.004429725 nan
e [ 49 120] m [  3  13  44  45  49  61  68 120]
e [ 49 120] m [  3  13  44  45  49  61  68 120]
e [ 49 120] m [  

m 28 0 0.008953513 nan
e [42 72] m [15 42 61 72]
e [42 72] m [15 42 61 72]
e [42 72] m [15 42 61 72]
e [42 72] m [15 42 61 72]
e [42 72] m [15 42 61 72]
e [42 72] m [15 42 61 72]
e [42 72] m [15 42 61 72]
m 29 4 0.008953513 0.005338513166754523
e [15 61] m [15 42 61 72]
e [15 61] m [15 42 61 72]
e [15 61] m [15 42 61 72]
e [15 61] m [15 42 61 72]
e [15 61] m [15 42 61 72]
e [15 61] m [15 42 61 72]
e [15 61] m [15 42 61 72]
m 30 4 0.008953513 0.004837851091569008
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 31 0 0.008953513 nan
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 32 0 0.008953513 nan
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 33 0 0.008953513 nan
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 34 0 0.008953513 nan
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 35 0 0.008953513 nan
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m [

...

m 547 25 0.0051781153 0.0062422253036626685
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 548 0 0.0051781153 nan
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 549 0 0.0051781153 nan
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 550 0 0.0051781153 nan
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 551 0 0.0051781153 nan
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 552 0 0.0051781153 nan
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 553 0 0.0051781153 nan
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 554 0 0.0051781153 nan
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 555 0 0.0051781153 nan
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 556 0 0.0051781153 nan
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 557 0 0.0051781153 nan
e [] m

m 598 24 0.0051781153 0.00385025688528503
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 599 0 0.0051781153 nan
e [  90 1088] m [  45   78   90  141  184  224  320  332  342  350  380  392  393  403
  515  728  783  793  940 1048 1049 1055 1056 1088]
e [  90 1088] m [  45   78   90  141  184  224  320  332  342  350  380  392  393  403
  515  728  783  793  940 1048 1049 1055 1056 1088]
e [  90 1088] m [  45   78   90  141  184  224  320  332  342  350  380  392  393  403
  515  728  783  793  940 1048 1049 1055 1056 1088]
e [  90 1088] m [  45   78   90  141  184  224  320  332  342  350  380  392  393  403
  515  728  783  793  940 1048 1049 1055 1056 1088]
e [  90 1088] m [  45   78   90  141  184  224  320  332  342  350  380  392  393  403
  515  728  783  793  940 1048 1049 1055 1056 1088]
e [  90 1088] m [  45   78   90  141  184  224  320  332  342  350  380  392  393  403
  515  728  783  793  940 1048 1049 1055 1056 1088]
e [  90 1088] m [  45   78   

m 33 0 0.006560957 nan
e [54 60] m [ 4 24 26 27 29 34 52 54 55 57 60 62 63 66 67]
e [54 60] m [ 4 24 26 27 29 34 52 54 55 57 60 62 63 66 67]
e [54 60] m [ 4 24 26 27 29 34 52 54 55 57 60 62 63 66 67]
e [54 60] m [ 4 24 26 27 29 34 52 54 55 57 60 62 63 66 67]
e [54 60] m [ 4 24 26 27 29 34 52 54 55 57 60 62 63 66 67]
e [54 60] m [ 4 24 26 27 29 34 52 54 55 57 60 62 63 66 67]
e [54 60] m [ 4 24 26 27 29 34 52 54 55 57 60 62 63 66 67]
m 34 15 0.006560957 0.009116014731777523
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 35 0 0.006560957 nan
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 36 0 0.006560957 nan
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 37 0 0.006560957 nan
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 38 0 0.006560957 nan
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 39 0 0.006560957 nan
e [17 25] m [ 6 17 22 25 31 59]
e [17 25] m [ 6 17 22 25 3

e [] m []
e [] m []
e [] m []
e [] m []
m 8 0 0.008094979 nan
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 9 0 0.008094979 nan
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 10 0 0.008094979 nan
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 11 0 0.008094979 nan
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 12 0 0.008094979 nan
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 13 0 0.008094979 nan
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 14 0 0.008094979 nan
e [14 30] m [14 19 30]
e [14 30] m [14 19 30]
e [14 30] m [14 19 30]
e [14 30] m [14 19 30]
e [14 30] m [14 19 30]
e [14 30] m [14 19 30]
e [14 30] m [14 19 30]
m 15 3 0.008094979 0.013293454196708403
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 16 0 0.008094979 nan
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 17 0 0.00809497

m 41 0 0.0038424605 nan
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 42 0 0.0038424605 nan
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 43 0 0.0038424605 nan
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 44 0 0.0038424605 nan
(45, 7, 78, 577)
e [ 8 26] m [ 8 26 35]
e [ 8 26] m [ 8 26 35]
e [ 8 26] m [ 8 26 35]
e [ 8 26] m [ 8 26 35]
e [ 8 26] m [ 8 26 35]
e [ 8 26] m [ 8 26 35]
e [ 8 26] m [ 8 26 35]
m 0 3 0.005375255 0.005527136899127855
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 1 0 0.005375255 nan
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 2 0 0.005375255 nan
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 3 0 0.005375255 nan
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 4 0 0.005375255 nan
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 5 0 0.005375255 nan
e [] m []
e [] m 

In [181]:
keys = np.asarray(list(itertools.product(*[list(range(len(gammas)))])))
reg_proj_norm = {}
nwloocv_leaf2_creleaf2 = {}
for sid in np.asarray(list(connectivity_data.structure_datas.keys())):
    reg_proj_norm[sid] = connectivity_data.structure_datas[sid].reg_proj_norm
    nwloocv_leaf2_creleaf2[sid] = connectivity_data.structure_datas[sid].loocv_predictions_leaf2_creleaf2

losses_leaf2_creleaf2 = get_loss(reg_proj_norm, nwloocv_leaf2_creleaf2,pred_ind = connectivity_data.creleaf2_evalindices, true_ind = connectivity_data.creleaf2_evalindices,keys = keys)
best_gamma_leaf2_creleaf2 = get_best_hyperparameters(losses_leaf2_creleaf2,keys)
meanloss_nw_leaf2_creleaf2 = get_loss_best_hyp(losses_leaf2_creleaf2, best_gamma_leaf2_creleaf2)

meanloss_nw_leaf2_creleaf2

0
1
2
3
4
5
6
7
8
9
10
11


array([0.06492248, 0.4966726 , 0.12877972, 0.23367772, 0.18424232,
       0.12156122, 0.19349936, 0.07775228, 0.25703143, 0.25605966,
       0.08174782, 0.31461163])

In [155]:
np.nanmean(connectivity_data.structure_datas[sid].loocv_predictions_leaf2_creleaf2)

0.0002715001673944673

In [2]:
connectivity_data.structure_datas[sid].loocv_predictions_creleaf2_creleaf2 

NameError: name 'connectivity_data' is not defined

In [183]:
#major_structure_ids
for sid in np.asarray(list(connectivity_data.structure_datas.keys())):
    connectivity_data.structure_datas[sid].loocv_predictions_creleaf2_creleaf2 = get_nwloocv_predictions_multimodel_merge_dists(connectivity_data.structure_datas[sid].reg_proj_norm, 
                                                                                       connectivity_data.structure_datas[sid].pairwise_distances,#np.ones(losses.shape),#(losses - np.nanmin(losses))**2,#**6, 
                                                                                       gammas, 
                                                                                       connectivity_data.creleaf2_index_matrices[sid], 
                                                                                       connectivity_data.creleaf2_index_matrices[sid])                                  

#meanloss_nw_leaf_leaf2wt
#meanloss_nw_wtleaf_leaf2wt

e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 0 0 0.003720867 nan
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 1 0 0.003720867 nan
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 2 0 0.003720867 nan
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 3 0 0.003720867 nan
e [ 7 31] m [ 7 31]
e [ 7 31] m [ 7 31]
e [ 7 31] m [ 7 31]
e [ 7 31] m [ 7 31]
e [ 7 31] m [ 7 31]
e [ 7 31] m [ 7 31]
e [ 7 31] m [ 7 31]
m 4 2 0.003720867 0.0023921207969033126
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 5 0 0.003720867 nan
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 6 0 0.003720867 nan
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 7 0 0.003720867 nan
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 8 0 0.003720867 nan
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 9 0 0.003720

/Users/samsonkoelle/anaconda3/envs/allen_010719_5/lib/python3.7/site-packages/ipykernel_launcher.py:68: RuntimeWarning: Mean of empty slice
/Users/samsonkoelle/anaconda3/envs/allen_010719_5/lib/python3.7/site-packages/ipykernel_launcher.py:31: RuntimeWarning: overflow encountered in exp


m 3 0 0.006630151 nan
e [3 5] m [3 5]
e [3 5] m [3 5]
e [3 5] m [3 5]
e [3 5] m [3 5]
e [3 5] m [3 5]
e [3 5] m [3 5]
e [3 5] m [3 5]
m 4 2 0.006630151 0.008498256389583791
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 5 0 0.006630151 nan
(6, 7, 7, 577)
e [ 58 107] m [ 58 107]
e [ 58 107] m [ 58 107]
e [ 58 107] m [ 58 107]
e [ 58 107] m [ 58 107]
e [ 58 107] m [ 58 107]
e [ 58 107] m [ 58 107]
e [ 58 107] m [ 58 107]
m 0 2 0.004429725 0.004187957008379939
e [ 39  74 116] m [ 39  74 116]
e [ 39  74 116] m [ 39  74 116]
e [ 39  74 116] m [ 39  74 116]
e [ 39  74 116] m [ 39  74 116]
e [ 39  74 116] m [ 39  74 116]
e [ 39  74 116] m [ 39  74 116]
e [ 39  74 116] m [ 39  74 116]
m 1 3 0.004429725 0.003056066631569228
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 2 0 0.004429725 nan
e [24 30] m [24 30]
e [24 30] m [24 30]
e [24 30] m [24 30]
e [24 30] m [24 30]
e [24 30] m [24 30]
e [24 30] m [24 30]
e [24 30] m [24 30]
m 3 2 0.004429725 

e [] m []
e [] m []
e [] m []
e [] m []
m 71 0 0.004429725 nan
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 72 0 0.004429725 nan
e [ 72 117 121] m [ 72 117 121]
e [ 72 117 121] m [ 72 117 121]
e [ 72 117 121] m [ 72 117 121]
e [ 72 117 121] m [ 72 117 121]
e [ 72 117 121] m [ 72 117 121]
e [ 72 117 121] m [ 72 117 121]
e [ 72 117 121] m [ 72 117 121]
m 73 3 0.004429725 0.007151720915007287
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 74 0 0.004429725 nan
e [27 73 89] m [27 73 89]
e [27 73 89] m [27 73 89]
e [27 73 89] m [27 73 89]
e [27 73 89] m [27 73 89]
e [27 73 89] m [27 73 89]
e [27 73 89] m [27 73 89]
e [27 73 89] m [27 73 89]
m 75 3 0.004429725 0.003941307482183275
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 76 0 0.004429725 nan
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 77 0 0.004429725 nan
e [83 90] m [83 90]
e [83 90] m [83 90]
e [83 90] m [83 90]
e [83 90] m [83 90]

e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 56 0 0.008953513 nan
e [ 6 32] m [ 6 32]
e [ 6 32] m [ 6 32]
e [ 6 32] m [ 6 32]
e [ 6 32] m [ 6 32]
e [ 6 32] m [ 6 32]
e [ 6 32] m [ 6 32]
e [ 6 32] m [ 6 32]
m 57 2 0.008953513 0.00745191762846182
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 58 0 0.008953513 nan
e [67 80] m [67 80]
e [67 80] m [67 80]
e [67 80] m [67 80]
e [67 80] m [67 80]
e [67 80] m [67 80]
e [67 80] m [67 80]
e [67 80] m [67 80]
m 59 2 0.008953513 0.007291197923902634
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 60 0 0.008953513 nan
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 61 0 0.008953513 nan
(62, 7, 85, 577)
e [782 855 953] m [782 855 953]
e [782 855 953] m [782 855 953]
e [782 855 953] m [782 855 953]
e [782 855 953] m [782 855 953]
e [782 855 953] m [782 855 953]
e [782 855 953] m [782 855 953]
e [782 855 953] m [782 855 953]
m 0 3 0.0051781153 0.005041590932178708
e [ 254 

...

m 532 2 0.0051781153 0.0024832791150224106
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 533 0 0.0051781153 nan
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 534 0 0.0051781153 nan
e [ 721 1045] m [ 721 1045]
e [ 721 1045] m [ 721 1045]
e [ 721 1045] m [ 721 1045]
e [ 721 1045] m [ 721 1045]
e [ 721 1045] m [ 721 1045]
e [ 721 1045] m [ 721 1045]
e [ 721 1045] m [ 721 1045]
m 535 2 0.0051781153 0.0036671772594503264
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 536 0 0.0051781153 nan
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 537 0 0.0051781153 nan
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 538 0 0.0051781153 nan
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 539 0 0.0051781153 nan
e [  5 734] m [  5 734]
e [  5 734] m [  5 734]
e [  5 734] m [  5 734]
e [  5 734] m [  5 734]
e [  5 734] m [  5 734]
e [  5 734] m [  5 734]
e [ 

m 597 0 0.0051781153 nan
e [  78  224  350  392 1049 1055] m [  78  224  350  392 1049 1055]
e [  78  224  350  392 1049 1055] m [  78  224  350  392 1049 1055]
e [  78  224  350  392 1049 1055] m [  78  224  350  392 1049 1055]
e [  78  224  350  392 1049 1055] m [  78  224  350  392 1049 1055]
e [  78  224  350  392 1049 1055] m [  78  224  350  392 1049 1055]
e [  78  224  350  392 1049 1055] m [  78  224  350  392 1049 1055]
e [  78  224  350  392 1049 1055] m [  78  224  350  392 1049 1055]
m 598 6 0.0051781153 0.0036547320179010764
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 599 0 0.0051781153 nan
e [  90 1088] m [  90 1088]
e [  90 1088] m [  90 1088]
e [  90 1088] m [  90 1088]
e [  90 1088] m [  90 1088]
e [  90 1088] m [  90 1088]
e [  90 1088] m [  90 1088]
e [  90 1088] m [  90 1088]
m 600 2 0.0051781153 0.004485420547406247
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 601 0 0.0051781153 nan
e [] m []
e [] m []
e [] m [

m 56 3 0.006560957 0.01549173320523092
(57, 7, 68, 577)
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 0 0 0.007270674 nan
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 1 0 0.007270674 nan
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 2 0 0.007270674 nan
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 3 0 0.007270674 nan
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 4 0 0.007270674 nan
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 5 0 0.007270674 nan
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 6 0 0.007270674 nan
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 7 0 0.007270674 nan
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 8 0 0.007270674 nan
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 9 0 0.007270674 nan
e [] m []
e [] m []
e []

e [] m []
m 11 0 0.008094979 nan
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 12 0 0.008094979 nan
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 13 0 0.008094979 nan
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 14 0 0.008094979 nan
e [14 30] m [14 30]
e [14 30] m [14 30]
e [14 30] m [14 30]
e [14 30] m [14 30]
e [14 30] m [14 30]
e [14 30] m [14 30]
e [14 30] m [14 30]
m 15 2 0.008094979 0.013577759418395254
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 16 0 0.008094979 nan
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 17 0 0.008094979 nan
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 18 0 0.008094979 nan
e [11 23] m [11 23]
e [11 23] m [11 23]
e [11 23] m [11 23]
e [11 23] m [11 23]
e [11 23] m [11 23]
e [11 23] m [11 23]
e [11 23] m [11 23]
m 19 2 0.008094979 0.006877588010647095
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []

m 30 2 0.0038424605 0.0035747421687340702
e [ 1  7 16 22 37 40 56 58] m [ 1  7 16 22 37 40 56 58]
e [ 1  7 16 22 37 40 56 58] m [ 1  7 16 22 37 40 56 58]
e [ 1  7 16 22 37 40 56 58] m [ 1  7 16 22 37 40 56 58]
e [ 1  7 16 22 37 40 56 58] m [ 1  7 16 22 37 40 56 58]
e [ 1  7 16 22 37 40 56 58] m [ 1  7 16 22 37 40 56 58]
e [ 1  7 16 22 37 40 56 58] m [ 1  7 16 22 37 40 56 58]
e [ 1  7 16 22 37 40 56 58] m [ 1  7 16 22 37 40 56 58]
m 31 8 0.0038424605 0.0022780031112521098
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 32 0 0.0038424605 nan
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 33 0 0.0038424605 nan
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 34 0 0.0038424605 nan
e [21 67] m [21 67]
e [21 67] m [21 67]
e [21 67] m [21 67]
e [21 67] m [21 67]
e [21 67] m [21 67]
e [21 67] m [21 67]
e [21 67] m [21 67]
m 35 2 0.0038424605 0.009973616580507903
e [13 18 49 51 64] m [13 18 49 51 64]
e [13 18 49 51 64] m [1

In [184]:
keys = np.asarray(list(itertools.product(*[list(range(len(gammas)))])))
reg_proj_norm = {}
nwloocv_creleaf2_creleaf2 = {}
for sid in np.asarray(list(connectivity_data.structure_datas.keys())):
    reg_proj_norm[sid] = connectivity_data.structure_datas[sid].reg_proj_norm
    nwloocv_creleaf2_creleaf2[sid] = connectivity_data.structure_datas[sid].loocv_predictions_creleaf2_creleaf2

losses_creleaf2_creleaf2 = get_loss(reg_proj_norm, nwloocv_creleaf2_creleaf2,pred_ind = connectivity_data.creleaf2_evalindices, true_ind = connectivity_data.creleaf2_evalindices,keys = keys)
best_gamma_creleaf2_creleaf2 = get_best_hyperparameters(losses_creleaf2_creleaf2,keys)
meanloss_nw_creleaf2_creleaf2 = get_loss_best_hyp(losses_creleaf2_creleaf2, best_gamma_creleaf2_creleaf2)

meanloss_nw_creleaf2_creleaf2

0
1
2
3
4
5
6
7
8
9
10
11


array([0.04951634, 0.4966726 , 0.10827024, 0.25932422, 0.15073582,
       0.11942033, 0.23823622, 0.0798464 , 0.2385843 , 0.19568441,
       0.06782042, 0.57384959])

In [1]:
meanloss_nw_creleaf2_creleaf2

NameError: name 'meanloss_nw_creleaf2_creleaf2' is not defined

In [23]:
# np.nanmean(get_nwloocv_predictions_multimodel_merge_dists(connectivity_data.structure_datas[sid].reg_proj_norm, 
#                                                                                        connectivity_data.structure_datas[sid].pairwise_distances,#np.ones(losses.shape),#(losses - np.nanmin(losses))**2,#**6, 
#                                                                                        gammas, 
#                                                                                        connectivity_data.leaf2_index_matrices[sid], 
#                                                                                        connectivity_data.creleaf2_index_matrices[sid])                                  
# )

In [ ]:
def get_indices2(firstlist, firstlisttest, secondlist):
    
    sl_unique = np.unique(secondlist)
    output = np.zeros((len(sl_unique), len(secondlist)), dtype = int)
    for i in range(len(sl_unique)):
        output[i,np.intersect1d(np.where(np.isin(firstlist,firstlisttest))[0], np.where(secondlist == sl_unique[i])[0])] = 1
    return(output)

In [151]:

def combine_predictions(predictions, eval_index_matrix):
    nmodels, ngammas, nexp, ntargets = predictions.shape
    combined_predictions = np.empty((ngammas, nexp, ntargets))
    for m in range(nmodels):
        combined_predictions[:, np.where(eval_index_matrix[m] == 1)[0]] = predictions[m][:,
                                                                          np.where(eval_index_matrix[m] == 1)[0]]

    return (combined_predictions)


def get_nwloocv_predictions_singlemodel_dists(projections, dists, gamma, model_indices, eval_indices):
    eval_index_val = np.where(eval_indices == 1)[0]
    model_index_val = np.where(model_indices == 1)[0]
    print('e',eval_index_val,'m',model_index_val)
    
    projections = np.asarray(projections, dtype=np.float32)

    nmod_ind = len(model_index_val)
    neval = len(eval_index_val)
    # nexp = centroids.shape[0]
    predictions = np.empty(projections.shape)
    predictions[:] = np.nan
    if len(model_index_val) > 0 and len(eval_index_val) > 0:
        # weights = np.exp(-dists[model_index_val][:, eval_index_val] / gamma)#np.exp(-dists[model_index_val] / gamma) #get_weights(centroids, gamma)
        for i in range(neval):
            matchindex = np.where(model_index_val == eval_index_val[i])[0]
            otherindices = np.setdiff1d(np.asarray(list(range(nmod_ind))), matchindex)
            # this order of operations is the fastest I found
            dists_i = dists[model_index_val][:, eval_index_val[i]] - np.min(
                dists[model_index_val[otherindices]][:, eval_index_val[i]])
            weights_i = np.exp(-dists_i * gamma)  # weights[i,:] / np.nansum(weights[i,:][otherindices])
            # print(np.nansum(weights[:,i][otherindices]))
            weights_i[matchindex] = 0
            weights_i = np.asarray(weights_i, dtype=np.float32)
            weights_i = weights_i / np.sum(weights_i)
            # weights_i[np.isnan(weights_i)] = 0.
            pred = np.dot(weights_i, projections[model_index_val])
            predictions[eval_index_val[i]] = pred

    return (predictions)


def get_nwloocv_predictions_multimodel_merge_dists(projections, dists, gammas, model_index_matrix, eval_index_matrix):
    predictions_unmerged = get_nwloocv_predictions_multimodel_dists(projections, dists, gammas, model_index_matrix,
                                                                    eval_index_matrix)
    print(predictions_unmerged.shape)
    predictions_merged = combine_predictions(predictions_unmerged, eval_index_matrix)

    return (predictions_merged)


def get_nwloocv_predictions_multimodel_dists(projections, dists, gammas, model_index_matrix, eval_index_matrix):
    ntargets = projections.shape[1]
    nexp = projections.shape[0]
    nmodels = model_index_matrix.shape[0]
    ngammas = len(gammas)

    projections = np.asarray(projections, dtype=np.float32)
    predictions = np.empty((nmodels, ngammas, nexp, ntargets))
    predictions[:] = np.nan
    for m in range(nmodels):
        
        #print()
        predictions[m] = np.asarray([get_nwloocv_predictions_singlemodel_dists(projections, dists, gammas[g],
                                                                               model_index_matrix[m],
                                                                               eval_index_matrix[m]) for g in
                                     range(ngammas)])
        print('m', m, len(np.where(model_index_matrix[m] ==1)[0]), np.nanmean(projections), np.nanmean(predictions[m]))

    return (predictions)


In [29]:
sid  = 512
np.nanmean(get_nwloocv_predictions_multimodel_dists(connectivity_data.structure_datas[sid].reg_proj_norm, connectivity_data.structure_datas[sid].pairwise_distances,#np.ones(losses.shape),#(losses - np.nanmin(losses))**2,#**6, 
                                                                                       gammas, 
                                                                                       connectivity_data.leaf2_index_matrices[sid], 
                                                                                       connectivity_data.creleaf2_index_matrices[sid]))

e [] m [ 0 12 16]
e [] m [ 0 12 16]
e [] m [ 0 12 16]
e [] m [ 0 12 16]
e [] m [ 0 12 16]
e [] m [ 0 12 16]
e [] m [ 0 12 16]
m 0 3 0.003720867 nan
e [] m [ 6 28]
e [] m [ 6 28]
e [] m [ 6 28]
e [] m [ 6 28]
e [] m [ 6 28]
e [] m [ 6 28]
e [] m [ 6 28]
m 1 2 0.003720867 nan
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 2 0 0.003720867 nan
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 3 0 0.003720867 nan
e [ 7 31] m []
e [ 7 31] m []
e [ 7 31] m []
e [ 7 31] m []
e [ 7 31] m []
e [ 7 31] m []
e [ 7 31] m []
m 4 0 0.003720867 nan
e [] m [ 9 19]
e [] m [ 9 19]
e [] m [ 9 19]
e [] m [ 9 19]
e [] m [ 9 19]
e [] m [ 9 19]
e [] m [ 9 19]
m 5 2 0.003720867 nan
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
e [] m []
m 6 0 0.003720867 nan
e [] m [ 5  7 14 15 30 31]
e [] m [ 5  7 14 15 30 31]
e [] m [ 5  7 14 15 30 31]
e [] m [ 5  7 14 15 30 31]
e [] m [ 5  7 14 15 30 31]
e [] m [ 5  7 14 15 30 31]
e [] m [ 5  7 14 15 30 31]
m 7 6 

/Users/samsonkoelle/anaconda3/envs/allen_010719_5/lib/python3.7/site-packages/ipykernel_launcher.py:58: RuntimeWarning: Mean of empty slice
/Users/samsonkoelle/anaconda3/envs/allen_010719_5/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: Mean of empty slice
  """


nan

In [31]:
sid = 512
print(np.nanmax(connectivity_data.structure_datas[sid].loocv_predictions_leaf2_creleaf2))
print(connectivity_data.structure_datas[sid].reg_proj_norm.max())
print(connectivity_data.structure_datas[sid].pairwise_distances.max())
#pconnectivity_data.structure_datas[sid].pairwise_distances.max()
print(connectivity_data.leaf2_index_matrices[sid].shape)
print(connectivity_data.creleaf2_index_matrices[sid].shape)
print(connectivity_data.leaf2_index_matrices[512].max())
print(connectivity_data.creleaf2_index_matrices[sid].max())

0.9880462288856506
0.9995457988579207
2279.780027200606
(13, 36)
(34, 36)
1
1


0


array([0.06492248])

In [ ]:

losses_leaf_leaf2 = get_loss(reg_proj_vcount_norm_renorms, nwloocv_leaf_leaf2,pred_ind = eval_indices_creleaf2ormore, true_ind = eval_indices_creleaf2ormore,keys = keys)
best_gamma_leaf_leaf2 = get_best_hyperparameters(losses_leaf_leaf2,keys)
meanloss_nw_leaf_leaf2 = get_loss_best_hyp(losses_leaf_leaf2, best_gamma_leaf_leaf2)


In [33]:
np.nanmax(get_nwloocv_predictions_multimodel_merge_dists(connectivity_data.structure_datas[sid].reg_proj_norm, 
                                               connectivity_data.structure_datas[sid].pairwise_distances,#np.ones(losses.shape),#(losses - np.nanmin(losses))**2,#**6, 
                                               gammas, 
                                               connectivity_data.creleaf2_index_matrices[sid], 
                                               connectivity_data.creleaf2_index_matrices[sid]))

(13, 7, 36, 577)


0.9880462288856506

In [37]:
#eval_indices_creleaf2ormore = get_eval_indices(indices_creleaf2ormore)

In [38]:
connectivity_data.creleaf2_index_matrices[512].max()

1

0


In [40]:
meanloss_nw_leaf_leaf2distscreleaf

array([2.])

In [41]:
nwloocv_leaf_leaf2[512].max()

0.0

In [86]:
# indices_leaf = {}
# #indices_wtinleaf = {}
# #indices_wtleaf = {}
# #indices_summary = {}
# #indices_summaryinleaf = {}
# # indices_major = {}
# # indices_majorinleaf = {}
# # indices_leaf2ormore = {}
# # indices_wtinleaf2ormore = {}
# # indices_leaf_reduced = {}
# # indices_wtinleaf_reduced = {}
# # indices_wt_leaf2ormore = {}
# # indices_leaf2ormore_wt = {}
# # indices_wt1ormore_leaf2ormore = {}
# # indices_leaf2ormore_wt1ormore = {}
# # indices_creleaf = {}
# # indices_creleaf2ormore = {}
# # creleafs = {}
# # creleafs_merged = {}
# for sid in np.asarray(list(connectivity_data.structure_datas.keys())):
    

#     #get the indices of experiments sharing leafs (nmodels is number of leafs)
#     indices_leaf[sid] = get_indices(leafs[sid]) #eval_indices
#     indices_leaf2ormore[sid] = screen_index_matrices(indices_leaf[sid], indices_leaf[sid])

#     #indices_creleaf = get_indices(leafs[sid])
    
#     #get the indices of the wts in the leaf (nmodels is number of leafs)
#     indices_wtinleaf[sid] = get_indices2(connectivity_data.creline[sid], np.asarray(['C57BL/6J']),leafs[sid]) #model_indices
    
#     #get indices of experiments sharing summary structure x cre combination (nmodel is number of cre x leaf combinations)
#     #indices_wtleaf[sid] = get_indices(creleafs_merged[sid])
    
#     #get indices of experiments sharing summary structure(nmodel is number of summary structures)
#     #indices_summary[sid] = get_indices(experiments_minor_structures[sid])
    
#     #get indices of experiments sharing major structure(nmodel is number of summary structures)
#     #indices_major[sid] = np.ones((1,experiments_minor_structures[sid].shape[0]))
    
#     #get indices of experiments sharing same major structure as a leaf (nmodel is number of leafs)
    
#     #get indices of experiments in same summary structure as a leaf (nmodel is number of leafs)
#     #indices_summaryinleaf[sid] = get_indices_summaryinleaf(experiments_minor_structures[sid], leafs[sid])
    
#     #evaluate models on leafs
    
# #    indices_wt1ormore_leaf2ormore[sid], indices_leaf2ormore_wt1ormore[sid] = screen_index_matrices3(indices_wtinleaf[sid], indices_leaf2ormore[sid])
    
# #    indices_wtinleaf2ormore[sid] = screen_index_matrices(indices_wtinleaf[sid], indices_wtinleaf[sid])
# #    indices_leaf_reduced[sid], indices_wtinleaf_reduced[sid]  = screen_index_matrices2( indices_leaf2ormore[sid],indices_wtinleaf[sid])


#     indices_creleaf[sid] = get_indices(np.asarray(creleafs_merged[sid]))
#     indices_creleaf2ormore[sid] = screen_index_matrices(indices_creleaf[sid], indices_creleaf[sid])
#     indices_majorinleaf[sid] = np.ones(indices_creleaf2ormore[sid].shape)#np.ones((len(np.unique(leafs[sid])),leafs[sid].shape[0]))#get_indices2(np.ones(len(leafs[sid])), np.asarray([1]),leafs[sid]) #model_indices

    


In [49]:
connectivity_data.creleaf2_evalindices

{512: array([ 7, 11, 21, 31])}

In [ ]:
losses_leaf_leaf2

In [100]:
reg_proj_norm[512].shape

(36, 577)

In [99]:
nwloocv_leaf_leaf2[512][0]

(7, 36, 577)

In [116]:
nwloocv_leaf_leaf2[512].max()

0.9990918040145189

In [118]:
losses_leaf_leaf2

{512: array([[0.03335401, 0.01796942, 0.0801405 , 0.05930302],
        [2.        , 2.        , 2.        , 2.        ],
        [2.        , 2.        , 2.        , 2.        ],
        [2.        , 2.        , 2.        , 2.        ],
        [2.        , 2.        , 2.        , 2.        ],
        [2.        , 2.        , 2.        , 2.        ],
        [2.        , 2.        , 2.        , 2.        ]])}

In [117]:
meanloss_nw_leaf_leaf2distscreleaf

array([0.04769174])